In [ ]:
from pennylane import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt
import qiskit
from qiskit import *
from qiskit import QuantumCircuit, ClassicalRegister
from azure.quantum.qiskit import AzureQuantumProvider
import sklearn.decomposition
from qiskit import transpile
from sklearn import metrics
from mitiq.interface import convert_to_mitiq, convert_from_mitiq


np.set_printoptions(threshold=np.inf)


provider = AzureQuantumProvider(
  resource_id="/subscriptions/.../resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/Projectname",
  location="westeurope"
)


In [ ]:
num_qubits = 3 


dev = qml.device("default.qubit", wires=num_qubits) 

In [ ]:
np.set_printoptions(threshold=np.inf)


data1 = np.loadtxt("TDS.txt") # import pandas as pd

data2 = np.loadtxt("TLD.txt") #

data3 = np.loadtxt("VDS.txt") #

data4 = np.loadtxt("VLD.txt") #

y_train= data2[:,] 

X_train2=data1[:]

y_test= data4[:,] 

X_test2=data3[:]



pi = np.pi


X_train1 = sklearn.preprocessing.normalize(X_train2, norm='l2',axis=0)

X_test1 = sklearn.preprocessing.normalize(X_test2, norm='l2', axis=0)

X_train = sklearn.preprocessing.normalize(X_train1, norm='l2',axis=1)

X_test = sklearn.preprocessing.normalize(X_test1, norm='l2', axis=1)


In [ ]:
x1_train=[]
x2_train=[]

for i in range(len(X_train)):
    if y_train[i]==1:
        a=X_train[i]
        x1_train.append(a)
    else:
        b=X_train[i]
        x2_train.append(b)

In [ ]:
num_qubits = 3
dev = qml.device('default.qubit', wires=num_qubits)

@qml.qnode(dev)
def circuit(params):
    qml.RY(params[0], wires=[0])
    qml.RY(params[1], wires=[1])
    qml.RY(params[2], wires=[2])
    qml.CNOT(wires=[0, 1])
    qml.RY(-params[1], wires=[1])
    qml.CNOT(wires=[0, 2])
    qml.RY(params[3], wires=[2])
    qml.CNOT(wires=[1, 2])
    qml.RY(params[4], wires=[0])
    qml.RY(params[5], wires=[1])
    qml.RY(params[6], wires=[2])
    return qml.state()

def loss(a,prediction):
    total_loss = np.real(np.sum(np.array((a-prediction)**2)))
    return total_loss

def cost(a,params):
    prediction = circuit(params)
    cost = loss(a,prediction)
    return cost


opt = qml.GradientDescentOptimizer(stepsize=0.1)

params = np.array([0.1,0.1,0.1,0.1,0.1,0.1,0.1],requires_grad=True)

y = []
for j in x1_train:
    for i in range(200):
        params_and_a,prev_cost = opt.step_and_cost(cost,j,params)
        params = params_and_a[1]
    y.append(params)

In [ ]:
y1 = []
params1 = np.array([0.1,0.1,0.1,0.1,0.1,0.1,0.1],requires_grad=True)
for j in x2_train:
    for i in range(200):
        params_and_a,prev_cost = opt.step_and_cost(cost,j,params1)
        params1 = params_and_a[1]
    y1.append(params1)

In [ ]:
y2 = []
params2 = np.array([0.1,0.1,0.1,0.1,0.1,0.1,0.1],requires_grad=True)
for j in X_test:
    for i in range(200):
        params_and_a,prev_cost = opt.step_and_cost(cost,j,params2)
        params2 = params_and_a[1]
    y2.append(params2)


In [ ]:
def statepreparation(X, circuit):
    
    circuit.ry(X[0], 0)
    circuit.ry(X[1], 1)
    circuit.ry(X[2], 2)
    circuit.cnot(0, 1)
    circuit.ry(-X[1], 1)
    circuit.cnot(0, 2)
    circuit.ry(X[3], 2)
    circuit.cnot(1, 2)
    circuit.ry(X[4], 0)
    circuit.ry(X[5], 1)
    circuit.ry(X[6], 2)
    

    return circuit

In [ ]:
simulator_backend = provider.get_backend("ionq.simulator") # for IonQ simulator
#simulator_backend = provider.get_backend("ionq.qpu") # for the 11-qubit IonQ quantum device


def classifer(X1, X2, shots):
    
    
        
    c = ClassicalRegister(1)
    circuit1 = QuantumCircuit(3)
    circuit1 = statepreparation(X1, circuit1)
    custom_gate1 = circuit1.to_gate()
    custom_gate11 = custom_gate1.control(1)
    
    circuit2 = QuantumCircuit(3)
    circuit2 = statepreparation(X2, circuit2)
    custom_gate2 = circuit2.to_gate()
    custom_gate22 = custom_gate2.control(1)
    
    circuit3 = QuantumCircuit(4, 1)
    
    circuit3.h(0)
    circuit3.append(custom_gate11, [0, 1, 2, 3])
    circuit3.x(0)
    
    circuit3.append(custom_gate22, [0, 1, 2, 3])
    circuit3.x(0)
    circuit3.h(0)
    
    circuit3.measure(0, 0)
    
    decomposed_circuit = qiskit.transpile(circuit3, basis_gates = ['x', 'h', 'rx','ry' , "cx"])
    
    cirq_circuit, native_type = convert_to_mitiq(decomposed_circuit)
    qiskit_circuit = convert_from_mitiq(cirq_circuit, native_type)
    
    result = execute(qiskit_circuit,simulator_backend,shots=shots).result()
    counts = result.get_counts(qiskit_circuit)
    result=np.zeros(2)
    for key in counts:
        result[int(key,2)]=counts[key]
    result/=shots
    
    return result[0]


In [ ]:
X1 = [[0.19539425, 0.33214242, 0.48356041, 0.43662365, 1.34293878,
        1.7338054 , 0.31279503],  [ 0.38959375, -0.25685394,  0.24199728,  0.75241645,  0.72382242,
         2.11590894,  0.08965113],  [ 0.07500809, -0.26711409,  0.3641704 ,  0.97253695,  1.51481631,
         1.62483754,  0.24954603],  [ 0.35757355, -0.46334231,  0.18351778,  1.28774761,  1.23639023,
         2.01913758,  0.03899236],  [ 0.37866811, -0.16640236,  0.11064104,  0.84041303,  1.30468286,
         1.85810968, -0.02075706],  [ 0.26547669, -0.23952756,  0.17319008,  1.14880967,  1.32040047,
         1.81578912,  0.04109467],  [ 0.21739227, -0.23859036,  0.11488936,  1.23254131,  1.23008166,
         1.86461883, -0.02705392],  [-0.03614929, -0.23571808,  0.18411019,  1.32876599,  1.66029564,
         1.51443252,  0.05139468],  [ 0.18976111, -0.21270956,  0.10251707,  1.22213856,  1.35191798,
         1.77982334, -0.03565001],  [ 0.22518876, -0.10339485,  0.12051604,  1.06221089,  1.38791388,
         1.76396529, -0.01514429],  [ 0.14273061, -0.04796028,  0.24025148,  0.98397631,  1.57602687,
         1.61976775,  0.10761306],  [ 0.16243248, -0.16516602,  0.23715548,  1.19799182,  1.28519107,
         1.83839833,  0.10368385],  [-0.05604973, -0.11155697,  0.25665113,  1.20842327,  1.48126508,
         1.46877315,  0.12280173],  [-0.04145449, -0.07718273,  0.2694197 ,  1.22582912,  1.57822802,
         1.50007137,  0.13414393],  [-0.17063479,  0.01543085,  0.35722331,  1.36946933,  1.83720747,
         1.42426376,  0.21809518],  [-0.10014041,  0.10520255,  0.34086502,  1.25376618,  1.5734316 ,
         1.54367368,  0.20286529],  [-0.16107382,  0.06714496,  0.24698993,  1.42628218,  1.92035325,
         1.32162172,  0.11233225],  [ 0.37392108,  0.06750886,  0.08181135,  0.92301243,  1.31982586,
         1.87691429, -0.05098139],  [0.16397258, 0.09624476, 0.18648821, 1.0082247 , 1.37397055,
        1.76536529, 0.05265763],  [0.2595915 , 0.17071684, 0.14487847, 0.83522178, 1.33311992,
        1.80567118, 0.01054185],  [0.06679517, 0.11934796, 0.30507486, 1.06128276, 1.42047753,
        1.68686936, 0.16599888],  [ 0.34220447, -0.05008331,  0.22562993,  0.79581308,  1.41659916,
         1.73315026,  0.08525588],  [ 0.18781501, -0.14184178,  0.2592282 ,  1.01358355,  1.34762309,
         1.78722153,  0.11893232],  [-0.11911945, -0.05450939,  0.43715585,  1.19871294,  1.77801321,
         1.45304544,  0.29370464],  [-0.01077642, -0.04412641,  0.36054098,  1.10995346,  1.58163246,
         1.63012698,  0.21758208],  [ 0.36948855, -0.01793767,  0.18363997,  0.7996713 ,  1.34657232,
         1.84864561,  0.03931194],  [-0.18117167,  0.08744113,  0.44756398,  1.1449805 ,  1.75636302,
         1.4235008 ,  0.30630254],  [-0.13404507,  0.1004662 ,  0.32237184,  1.2277674 ,  1.77249608,
         1.40366937,  0.18182589],  [0.03749256, 0.11347404, 0.22817896, 1.1063507 , 1.65355101,
        1.51105593, 0.08718721],  [-0.13433544,  0.12509189,  0.29705444,  1.31094965,  1.7647881 ,
         1.40721691,  0.15731382],  [0.09821822, 0.06107421, 0.15134073, 1.1403769 , 1.41030979,
        1.56587979, 0.00924534],  [0.29521613, 0.00809308, 0.16930936, 0.92443418, 1.4076533 ,
        1.76934021, 0.02607225],  [-0.15174122,  0.1270131 ,  0.34260306,  1.14757626,  1.63552862,
         1.3904718 ,  0.2023812 ],  [-0.358606  ,  0.3193096 ,  0.13295789,  1.71828018,  1.92474547,
         1.13598187,  0.01417528],  [ 0.22337814,  0.05190814, -0.01926951,  1.62375491,  1.18156323,
         1.88648142, -0.14972929],  [-0.01099699,  0.04190353,  0.0083727 ,  1.65380947,  1.51211002,
         1.51661401, -0.12376187],  [-0.14969335,  0.2129735 ,  0.2279069 ,  1.69209591,  1.66507238,
         1.50745936,  0.10263751],  [-0.07925548,  0.2540749 ,  0.13393219,  1.61802347,  1.75843109,
         1.48442836,  0.01300492],  [ 0.31956707, -0.05420745, -0.04058529,  1.42534344,  1.08296857,
         1.95980794, -0.13756307],  [ 0.04317768, -0.11082708,  0.0484837 ,  1.51740394,  1.59069413,
         1.55270771, -0.04271348],  [ 0.22895048, -0.17017377, -0.01030084,  1.63101153,  1.21446651,
         1.9266245 , -0.11213258],  [ 0.21114586, -0.10415443, -0.04243839,  1.34912561,  1.37811036,
         1.79831824, -0.13669562],  [ 0.21785632, -0.2205074 ,  0.02199037,  1.47200552,  1.30801131,
         1.79245901, -0.07641055],  [ 0.21891039, -0.25591923,  0.0311243 ,  1.57605913,  1.34845478,
         1.83254585, -0.07049245],  [ 0.13570281, -0.40199933, -0.00347677,  1.47473102,  1.4181032 ,
         1.53588178, -0.10144535],  [ 0.22414718, -0.31419587,  0.00458243,  1.34622458,  1.28521942,
         1.80336078, -0.10378816],  [-0.27384662,  0.07845316,  0.22486016,  1.60822866,  1.81065546,
         1.29378821,  0.10235694],  [ 0.10815006,  0.07295327,  0.0326984 ,  1.37514574,  1.37999631,
         1.68251335, -0.09324391],  [ 0.07074465,  0.04035501,  0.0790045 ,  1.38996094,  1.53980582,
         1.55092347, -0.04810811],  [-0.03447757,  0.06021729,  0.14737042,  1.44858783,  1.51524796,
         1.54536084,  0.02042789],  [-0.13956286,  0.14441729,  0.20767418,  1.51972268,  1.7540173 ,
         1.41417674,  0.08474347],  [-0.01068739,  0.17788564,  0.14638139,  1.43804236,  1.44701937,
         1.59406408,  0.02082809],  [-0.1072549 ,  0.19504906,  0.3199601 ,  1.50659752,  1.87793289,
         1.509998  ,  0.20169966],  [0.09242646, 0.13291679, 0.12623382, 1.29989847, 1.46462526,
        1.55716589, 0.0055727 ],  [-0.16152339,  0.17204319,  0.24737095,  1.45247253,  1.78877696,
         1.42928195,  0.13102271]]




In [ ]:
X2 = [[0.33041899, 0.23114166, 0.37438005, 0.49143933, 1.24903594,
        1.8313334 , 0.16238361],  [0.45735402, 0.13419632, 0.26240997, 0.37485281, 1.22799339,
        1.85087431, 0.04238257],  [ 0.43821719, -0.58285539,  0.26564474,  0.96471855,  0.80309688,
         2.3208112 ,  0.04270536],  [ 0.26053485, -0.5834416 ,  0.26821655,  1.13882426,  0.6391462 ,
         2.2346121 , -0.07224293],  [ 0.17188791, -0.45174934,  0.2251163 ,  1.21047384,  1.40799853,
         1.78869258, -0.05625739],  [ 0.24749584, -0.41349444,  0.20348214,  1.39154638,  1.25405781,
         1.93200514, -0.11104563],  [ 0.29734969, -0.23094365,  0.14693383,  1.2192806 ,  1.32960135,
         1.8987965 , -0.17335546],  [ 0.12785351, -0.26595985,  0.20744407,  1.40265244,  1.33344308,
         1.76320497, -0.11363357],  [ 0.43274374, -0.14604803,  0.17385362,  0.88635634,  1.31400384,
         1.95894593, -0.15044943],  [ 0.33064177, -0.01050624,  0.21899436,  0.98357916,  1.22695332,
         1.86083754, -0.11540258],  [ 0.22811227, -0.05239112,  0.18973969,  1.08777966,  1.08453859,
         1.90765576, -0.14604056],  [ 0.21803712, -0.14445486,  0.18719744,  1.24150515,  1.09475137,
         1.92310532, -0.14959071],  [ 0.2307781 , -0.04411599,  0.24124634,  1.00907158,  1.49597843,
         1.76124455, -0.0834378 ],  [ 0.1479513 , -0.03570312,  0.27315662,  1.07926359,  1.36683568,
         1.76213141, -0.05279716],  [ 0.35465046, -0.07863972,  0.17429171,  0.91648153,  1.06997581,
         1.91794975, -0.15767291],  [ 0.20163007, -0.04897254,  0.18072466,  1.01778965,  1.07166844,
         1.85920488, -0.15265058],  [ 0.40583028,  0.02123089,  0.25378282,  0.80372447,  1.34224957,
         1.86857522, -0.07706712],  [ 0.36857539, -0.22796713,  0.21249458,  1.02045369,  1.08490395,
         2.02828297, -0.12662836],  [ 0.52570717, -0.21375379,  0.19838168,  0.60619442,  1.22709575,
         2.07907852, -0.13159739],  [ 0.39257084, -0.0910118 ,  0.24081161,  0.69701568,  1.08798281,
         1.83044713, -0.11407738],  [ 0.24062864, -0.07863104,  0.26901138,  0.89255358,  1.10911407,
         1.87485929, -0.08710476],  [ 0.57709447,  0.07936299,  0.28253227,  0.49105396,  1.39404017,
         1.93423627, -0.07022035],  [ 3.21880023e-01,  7.64956748e-04,  3.07088779e-01,
         7.01731058e-01,  1.28732738e+00,  1.86987680e+00,
        -4.36978839e-02],  [ 0.27550941, -0.11057966,  0.30218098,  0.9590869 ,  1.21311366,
         1.90326518, -0.04853742],  [ 0.27547383, -0.06358886,  0.23309843,  1.02330434,  1.14255724,
         1.89707005, -0.12400093],  [ 0.25670826, -0.1097758 ,  0.21576965,  1.11005982,  1.17900855,
         1.89710906, -0.14177876],  [ 0.46811515, -0.23791498,  0.21170581,  0.99382834,  1.28570112,
         1.95869256, -0.1522041 ],  [ 0.14045463, -0.22698745,  0.3156599 ,  1.21383927,  1.46985165,
         1.72705858, -0.03840837],  [ 0.06069365, -0.1983774 ,  0.29065465,  1.1387387 ,  1.53712337,
         1.61550555, -0.06325464],  [ 0.35461032, -0.087695  ,  0.28198325,  0.81740776,  1.41215569,
         1.85318942, -0.07494766],  [ 0.29073886, -0.10109651,  0.33293886,  0.76633181,  1.37623528,
         1.73337558, -0.02356631],  [ 0.37867471,  0.01676234,  0.28294321,  0.64674061,  1.22172363,
         1.88619866, -0.07855971],  [ 0.42367915, -0.03736284,  0.28958829,  0.64537535,  1.34823623,
         1.95353862, -0.07207833],  [ 0.46345749, -0.06086351,  0.24804832,  0.65753995,  1.15390722,
         1.95859607, -0.12106809],  [ 0.17453245, -0.1599039 ,  0.32196034,  1.07819746,  0.98306552,
         1.99613564, -0.05248097],  [-0.14641915,  0.05542538,  0.40609878,  1.04443166,  1.39244969,
         1.49969923,  0.02800254],  [ 0.29329001, -0.04629125,  0.20799299,  1.01866423,  0.93871855,
         1.96610804, -0.16972975],  [ 0.16605973, -0.16226654,  0.20259939,  1.21445587,  1.17502285,
         1.83178412, -0.17157384],  [ 0.21049985, -0.04506443,  0.18925624,  1.10877938,  1.23051103,
         1.82164616, -0.18506082],  [ 0.31985261,  0.09418307,  0.18678487,  0.84995502,  1.14009695,
         1.86628479, -0.18748971],  [ 0.35443468,  0.07975886,  0.23164147,  0.82918811,  1.12955347,
         1.87411367, -0.13978601],  [ 0.38981556,  0.05660062,  0.25076882,  0.64234303,  1.14475131,
         1.86889458, -0.12218039],  [ 0.57454574, -0.31274174,  0.23508811,  0.76659032,  1.32062862,
         2.09726046, -0.13692506],  [ 0.28037676, -0.07706055,  0.32085274,  0.72017633,  1.27945096,
         1.78910193, -0.07031692],  [ 0.46576934, -0.13947081,  0.28391395,  0.69058131,  1.31918528,
         1.96844736, -0.10202037],  [ 0.39937057, -0.14849414,  0.2839659 ,  0.79826715,  1.31222779,
         1.96256728, -0.10309927],  [ 0.20239627, -0.04030015,  0.38790787,  0.79729957,  1.4124046 ,
         1.7270838 ,  0.00201544],  [ 0.17024754, -0.06477902,  0.32835123,  0.83912792,  1.3038714 ,
         1.79264495, -0.05814325],  [ 2.81208885e-01,  9.13946924e-04,  2.72657520e-01,
         9.49293219e-01,  1.19243302e+00,  1.90109608e+00,
        -1.16617597e-01],  [ 0.11844101,  0.10026752,  0.354     ,  0.9692172 ,  1.54992014,
         1.66624838, -0.03510217],  [ 0.22472221,  0.20775394,  0.28314368,  0.87051094,  1.29602183,
         1.80434267, -0.09770407],  [ 0.18709758,  0.14210722,  0.33655812,  1.04867699,  1.31209429,
         1.82827964, -0.04849249],  [ 0.1980249 ,  0.11165764,  0.31200814,  1.01019178,  1.35683494,
         1.75844933, -0.07590197],  [ 0.33389613,  0.11525086,  0.2807162 ,  0.84421167,  1.30887663,
         1.86335509, -0.10674445]]


In [ ]:
X3 = [[0.15608126, 0.00309801, 0.33794568, 0.58686454, 0.57853045,
        2.19687207, 0.31653755], [1.55726693e-03, 2.04844196e-02, 4.67075713e-01, 7.33942163e-01,
        1.38942566e+00, 1.80954321e+00, 4.45192627e-01], [0.24956111, 0.10790119, 0.24213162, 0.63295215, 1.32340329,
        1.79127316, 0.22150017], [0.06305553, 0.0910472 , 0.39699685, 0.7648176 , 1.5598344 ,
        1.54912068, 0.37115658], [0.11006238, 0.02642571, 0.38224   , 0.79244947, 1.5264865 ,
        1.6380663 , 0.35510721], [0.1463689 , 0.0125664 , 0.26358926, 0.88524283, 1.27756211,
        1.85399897, 0.23715044], [0.09978387, 0.03279756, 0.27515199, 0.87368371, 1.50170882,
        1.62363241, 0.24830061], [-0.19854363,  0.08639061,  0.35280456,  1.18364102,  1.8558004 ,
         1.34624617,  0.32879973], [-0.01469843,  0.11843603,  0.2642275 ,  1.06518484,  1.4898289 ,
         1.6191363 ,  0.23706392], [0.33760205, 0.15179649, 0.12125441, 0.7273392 , 1.29210318,
        1.76384486, 0.095098  ], [-0.01083849,  0.14842149,  0.33821319,  0.93369648,  1.56847289,
         1.5012022 ,  0.30431788], [0.34163158, 0.08406227, 0.08490122, 0.70974667, 1.16054998,
        1.92180929, 0.05858356]]


In [ ]:
D1 = []


for x1 in X1:
    for x2 in X3:
        a = classifer(x1, x2, shots=1000)
        D1.append(a)


D2 = []

for x1 in X2:
    for x2 in X3:
        b = classifer(x1, x2, shots=1000)
        D2.append(b)


In [ ]:
D11 = np.reshape(D1, (len(x1_train), len(X_test)))
D22 = np.reshape(D2, (len(x2_train), len(X_test)))

d11 = np.max(D11, axis=0)
d22 = np.max(D22, axis=0)
m = len(X_test)

Prediction=[]
for i in range(m):
    if d11[i] >= d22[i]:
        Prediction.append(1)
    else:
        Prediction.append(0)
    
BB = metrics.confusion_matrix(y_test, Prediction, labels=[0,1])

print(Prediction)

print(BB)      